# 10강. 번역기를 만들어보자

## 10-7. 번역기 만들기 (1) 데이터 전처리

![데이터받아오기](https://user-images.githubusercontent.com/116326867/206612317-e9fb579f-6192-4dc5-9744-58cf060a9b6f.png)

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
93433,Do you plan to buy that car?,Prévoyez-vous de faire l'acquisition de cette ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
15237,Keep us covered.,Couvre-nous.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
153171,Will you drink another cup of coffee?,Voulez-vous une autre tasse de café ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
186185,"No matter how long it takes, I will finish the...","Peu importe le temps que ça prend, je finirai ...",CC-BY 2.0 (France) Attribution: tatoeba.org #3...
157851,Do you think you'd like to work for us?,Pensez-vous que vous aimeriez travailler avec ...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [3]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
32397,This is way better.,C'est bien mieux.
37572,My friend helped me.,Mon amie m'a aidée.
26750,Tom likes country.,Tom aime la country.
26012,That's undeniable.,C'est indéniable.
38457,There's room inside.,Il y a de la place à l'intérieur.


In [4]:
# 시작 토큰<sos>과 종료 토큰<eos> 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t ' + x + ' \n')
# fra 각 열에 앞 뒤로 각각 \t와 \n을 넣기 

print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
28009,You're courageous.,\t Tu es courageuse. \n
13328,Congratulations!,\t Toutes nos félicitations ! \n
34652,Do exactly as I say.,\t Fais précisément ce que je dis. \n
25122,It's a good movie.,\t C'est un bon film. \n
25801,Stand up straight.,\t Tenez-vous droite ! \n


- [apply(lambda x)](https://ordo.tistory.com/128)  

각 단어에 부여된 고유한 정수로 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친 후 단어장 만들기(영어와 프랑스어는 사용하는 언어가 다르므로 단어장을 별도로 만든다)

In [5]:
# 영어
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [19, 3, 8], [19, 3, 8]]

In [6]:
# 프랑스어
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[10, 1, 19, 5, 1, 31, 1, 11],
 [10, 1, 15, 5, 12, 16, 29, 2, 14, 1, 11],
 [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]

In [7]:
# 단어장 만들기
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73


In [8]:
# 영어 데이터와 프랑스어 데이터의 최대 길이(for 패딩)
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


In [9]:
# 전체적인 통계 정보
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


인코더의 입력으로 사용되는 영어 시퀀스와 달리, 프랑스어 시퀀스는 2가지 버전으로 나누어 준비해야 한다. 하나는 디코더의 출력(< sos > 필요 없음)과 비교해야 할 정답 데이터로 사용하고, 다른 하나는 교사 강요(Teacher forcing)를 위해 디코더의 입력(< eos > 필요 없음)으로 사용한다.

In [10]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [11]:
print(decoder_input[:3])
print(decoder_target[:3])
# 디코더 입력은 4(< eos > 토큰) 제거, 디코더 출력은 3(< sos > 토큰) 제거

[[10, 1, 19, 5, 1, 31, 1], [10, 1, 15, 5, 12, 16, 29, 2, 14, 1], [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 11], [1, 15, 5, 12, 16, 29, 2, 14, 1, 11], [1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]


In [12]:
# 패딩하기
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


In [13]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [14]:
# 각 정수에 대해서 벡터화 방법으로 원-핫 인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22, 53)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


- 원-핫 인코딩 후 데이터의 크기는 (샘플 수 x 샘플의 길이 x 단어장의 크기).

In [15]:
# 50000건 중 3000건 검증데이터로, 나머지를 학습데이터로
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 22, 53)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


## 10-8. 번역기 만들기 (2) 모델 훈련하기

In [16]:
# 인코더 설계
# 입력 텐서 생성(입력 문장을 저장하게 될 변수 텐서)
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성, hidden state와 cell state 리턴받을 수 있도록 함
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

- RNN의 경우 인코더의 마지막 hidden state를 디코더의 첫번째 hidden state로 사용한다. LSTM의 경우에는 cell state도 존재하기 때문에 인코더 LSTM 셀의 마지막 time step의 hidden state와 cell state를 디코더 LSTM의 첫번째 hidden state와 cell state로 전달해주어야 한다.

In [17]:
# 디코더 설계
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

# 디코더 출력층
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

# 인코더와 디코더 연결하기
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 53)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 317440      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [18]:
# 학습하기
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
         validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
         batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 31s 20ms/step - loss: 1.0020 - val_loss: 0.8798
Epoch 2/50
368/368 [==============================] - 6s 18ms/step - loss: 0.6329 - val_loss: 0.7185
Epoch 3/50
368/368 [==============================] - 7s 18ms/step - loss: 0.5371 - val_loss: 0.6447
Epoch 4/50
368/368 [==============================] - 7s 18ms/step - loss: 0.4889 - val_loss: 0.6030
Epoch 5/50
368/368 [==============================] - 6s 18ms/step - loss: 0.4554 - val_loss: 0.5675
Epoch 6/50
368/368 [==============================] - 7s 18ms/step - loss: 0.4294 - val_loss: 0.5451
Epoch 7/50
368/368 [==============================] - 7s 18ms/step - loss: 0.4070 - val_loss: 0.5163
Epoch 8/50
368/368 [==============================] - 7s 18ms/step - loss: 0.3881 - val_loss: 0.4953
Epoch 9/50
368/368 [==============================] - 7s 18ms/step - loss: 0.3709 - val_loss: 0.4764
Epoch 10/50
368/368 [==============================] - 7s 18ms/step - loss: 0.3557 - val_l

## 10-9. 번역기 만들기 (3) 모델 테스트하기

테스트 단계의 디코더 모델은 다시 설계를 해주어야 한다. 훈련시에는 학습해야 할 타겟 문장을 디코더 모델의 입력, 출력 시퀀스로 넣어 주고, 디코더 모델이 타겟 문장을 한꺼번에 출력하게 할 수 있는 반면, 테스트 단계에서는 그렇게 할 수가 없기 때문이다.  
하나의 문장을 만들어 내기 위해 루프를 돌며 단어를 하나씩 차례로 예측하면서, 예측된 단어가 다음 입력으로 재사용되는 과정을 반복해주어야 한다.  
1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state 얻기
2. < sos > 토큰인 \t를 디코더에 입력
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 하기
4. 3번을 반복하다가 < eos > 토큰인 \n이 예측되면 중단

In [19]:
# 인코더 정의
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 53)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 317440    
Total params: 317,440
Trainable params: 317,440
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 디코더 설계
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용
# 구체적인 동작 자체는 def decode_sequence()에 구현
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states = [state_h, state_c]

In [21]:
# 디코더 출력층 재설계
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]              

In [22]:
# 단어에서 정수로, 정수에서 단어로 바꾸는 사전 만들기(테스트 결과 해석을 위해서)
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [23]:
# decode_sequence() 함수만들기
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # < SOS >에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # < eos >에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

- [np.argmax](https://jimmy-ai.tistory.com/72) : 리스트 등의 원소 중 가장 큰 값의 인덱스를 반환한다. 가장 큰 원소가 여러개 있는 경우 가장 앞의 인덱스를 반환한다.  
- [np.zeros](https://cosmosproject.tistory.com/408)

In [24]:
# 출력 결과 테스트
for seq_index in [3,46,132,576,10001]: # 입력 문장의 인덱스(변경 가능)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  marche. 
-----------------------------------
입력 문장: Begin.
정답 문장:  Commence. 
번역기가 번역한 문장:  commencez. 
-----------------------------------
입력 문장: No way!
정답 문장:  C'est exclu ! 
번역기가 번역한 문장:  c'est impressible ! 
-----------------------------------
입력 문장: Have fun.
정답 문장:  Amuse-toi bien ! 
번역기가 번역한 문장:  amusez-vous bien ! 
-----------------------------------
입력 문장: Get off my car.
정답 문장:  Descends de ma voiture. 
번역기가 번역한 문장:  lâchez-moi ! 
